In [1]:
import requests
import json
import time
import os
import pandas as pd
import dlt
from dlt.sources.filesystem import filesystem
from dotenv import load_dotenv

In [2]:
# === CONFIGURATION ===
load_dotenv()
CLIENT_ID = os.getenv(f'CLIENT_ID')
CLIENT_SECRET = os.getenv(f'CLIENT_SECRET')

TOKENS_FILE = 'secrets/strava_tokens.json'

In [3]:
# === LOAD OR REFRESH TOKENS ===
def load_tokens():
    if os.path.exists(TOKENS_FILE):
        with open(TOKENS_FILE, 'r') as f:
            return json.load(f)
    else:
        raise FileNotFoundError("Token file not found. Authorize first and save your tokens.")

def save_tokens(tokens):
    with open(TOKENS_FILE, 'w') as f:
        json.dump(tokens, f)

def refresh_tokens(tokens):
    if time.time() > tokens['expires_at']:
        print("Access token expired. Refreshing...")
        response = requests.post("https://www.strava.com/oauth/token", data={
            'client_id': CLIENT_ID,
            'client_secret': CLIENT_SECRET,
            'grant_type': 'refresh_token',
            'refresh_token': tokens['refresh_token']
        })
        new_tokens = response.json()
        tokens.update({
            'access_token': new_tokens['access_token'],
            'refresh_token': new_tokens['refresh_token'],
            'expires_at': new_tokens['expires_at']
        })
        save_tokens(tokens)
    return tokens

# === GET ACTIVITIES ===
def get_activities(access_token, per_page=30):
    headers = {'Authorization': f"Bearer {access_token}"}
    page = 1

    while True:
        response = requests.get(
            'https://www.strava.com/api/v3/athlete/activities',
            headers=headers,
            params={'per_page': per_page, 'page': page}
        )
        data = response.json()

        if not data:
            break

        for activity in data:
            yield activity
        
        page += 1

In [4]:
# === MAIN FLOW ===
tokens = load_tokens()
tokens = refresh_tokens(tokens)
ACCESS_TOKEN=tokens['access_token']

activities_generator = get_activities(ACCESS_TOKEN)

activities = [activity for activity in activities_generator]

# === DISPLAY RESULTS ===
# for act in activities:
#     print(f"{act['start_date'][:10]} - {act['name']} - {act['distance']/1000:.2f} km")

In [5]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import OffsetPaginator

os.environ["ACCESS_TOKEN"] = ACCESS_TOKEN
@dlt.source
def strava_source(
    access_token=dlt.secrets.value
):
    client = RESTClient(
        base_url='https://www.strava.com/api/v3/',
        auth=BearerTokenAuth(token=access_token),
        paginator=OffsetPaginator(
            limit=100,
            limit_param='per_page',
            offset=1,
            offset_param='page',
            stop_after_empty_page=True,
            total_path=None
        )
    )

    @dlt.resource(
        write_disposition="replace",
        #primary_key="id",
    )
    def activities():
        for page in client.paginate("athlete/activities"):
            yield page

    return activities

    @dlt.resource(
        write_disposition="replace",
        #primary_key="id",
    )
    def athlete():
        for page in client.paginate("athlete"):
            yield page

    return athlete

In [6]:
pipeline_s3 = dlt.pipeline(
    pipeline_name="strava_to_s3",       # you can keep the same name if you like
    destination="filesystem",            # ← switch to filesystem
    dataset_name="strava_activities_s3"  # name for the S3 “folder” in your bucket
)
load_info_s3 = pipeline_s3.run(strava_source(), loader_file_format = "csv")
#print(load_info_s3)

In [7]:
pipeline_redshift = dlt.pipeline(
    pipeline_name="strava_s3_to_redshift",
    destination="redshift",
    dataset_name="strava_rest_api_dataset"
)

source_from_s3 = filesystem(
    bucket_url="s3://billy-heidel-test-bucket/strava_activities_s3",
    file_glob="**/*.csv"
)

# Run pipeline to move CSV data from S3 into Redshift
load_info_redshift = pipeline_redshift.run(source_from_s3)
#print("Loaded to Redshift:", load_info_redshift)

In [8]:
pipeline = dlt.pipeline(
    pipeline_name="strava_rest_api_redshift",
    destination="redshift",
)
#load_info = pipeline.run(strava_source())

In [9]:
# Define a dlt pipeline with automatic normalization
pipeline = dlt.pipeline(
    destination="duckdb",
    pipeline_name="strava_rest_api",
)

# run the pipeline with the new resource
load_info = pipeline.run(strava_source())

In [10]:
pipeline.dataset(dataset_type="default").schema.data_table_names()

['activities', 'activities__start_latlng', 'activities__end_latlng']

In [11]:
pipeline.dataset(dataset_type="default").activities.df()

,resource_state,athlete__id,athlete__resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,...,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,_dlt_load_id,_dlt_id,workout_type
0,2,148269563,1,Morning Walk,1166.9,897,897,9.7,Walk,Walk,...,15588803482,15588803482,garmin_ping_442780035147,False,0,0,False,1748357662.911686,h5F+Z1fgWJ2fhg,<NA>
1,2,148269563,1,Morning Walk,1422.2,938,938,14.5,Walk,Walk,...,15579558715,15579558715,garmin_ping_442500649377,False,0,0,False,1748357662.911686,Vf0QsiNoieA8Tg,<NA>
2,2,148269563,1,Morning Run,10322.1,3604,3604,0.0,Run,Run,...,15569576686,15569576686,garmin_ping_442179704735,False,0,0,False,1748357662.911686,+NkyQVWQLdguhg,<NA>
3,2,148269563,1,Afternoon Walk,3073.0,2257,2257,20.1,Walk,Walk,...,15562343689,15562343689,garmin_ping_441973417210,False,0,0,False,1748357662.911686,jaXXp0PXtgHFHw,<NA>
4,2,148269563,1,Evening Walk,2938.9,2295,2295,3.5,Walk,Walk,...,15552255364,15552255364,garmin_ping_441687539687,False,0,0,False,1748357662.911686,YxDN69Gbsf2ovg,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2,148269563,1,Evening Run,568.6,3321,3321,0.0,Run,Run,...,14524948604,14524948604,garmin_ping_410575511422,False,0,0,False,1748357662.911686,lgVo4eM//dOosw,<NA>
69,2,148269563,1,Morning Run,7129.3,2613,2613,0.0,Run,Run,...,13310815861,13310815861,stripped_garmin_ping_372336753957,False,0,0,False,1748357662.911686,t6/td0iUhC+0xQ,<NA>
70,2,148269563,1,Morning Run,5099.3,1533,1533,0.0,Run,Run,...,13310815879,13310815879,stripped_garmin_ping_372336753968,False,0,0,False,1748357662.911686,mgR/smeZ3RT76Q,<NA>
71,2,148269563,1,Morning Run,6040.3,1589,1589,0.0,Run,Run,...,13310818837,13310818837,stripped_garmin_ping_372336816902,False,0,0,False,1748357662.911686,5L3vT0c/AGNH3g,<NA>


In [12]:
import duckdb

conn = duckdb.connect(f"strava_rest_api.duckdb")
conn.sql(f"SET search_path = 'strava_rest_api'")
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,strava_rest_api,strava_rest_api_dataset,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,strava_rest_api,strava_rest_api_dataset,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,strava_rest_api,strava_rest_api_dataset,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,strava_rest_api,strava_rest_api_dataset,activities,"[resource_state, athlete__id, athlete__resourc...","[BIGINT, BIGINT, BIGINT, VARCHAR, DOUBLE, BIGI...",False
4,strava_rest_api,strava_rest_api_dataset,activities__end_latlng,"[value, _dlt_parent_id, _dlt_list_idx, _dlt_id]","[DOUBLE, VARCHAR, BIGINT, VARCHAR]",False
5,strava_rest_api,strava_rest_api_dataset,activities__start_latlng,"[value, _dlt_parent_id, _dlt_list_idx, _dlt_id]","[DOUBLE, VARCHAR, BIGINT, VARCHAR]",False


In [13]:
from dlt.sources.rest_api import RESTAPIConfig, rest_api_source

config: RESTAPIConfig = {
    "client": {
        "base_url": "https://www.strava.com/api/v3/",
        "auth": {
            "token": ACCESS_TOKEN, # <--- we already configured access_token above
        },
        "paginator": OffsetPaginator(
            limit=100,
            limit_param='per_page',
            offset=1,
            offset_param='page',
            stop_after_empty_page=True,
            total_path=None
        )
    },
    "resources": [  # <--- list resources
        {
            "name": "activities",
            "endpoint": {
                "path": "athlete/activities",
            },
            "write_disposition": "replace",  # ✅ Merge mode
            #"primary_key": "id" 
        },
    ],
}

strava_source = rest_api_source(config)


pipeline_from_config = dlt.pipeline(
    pipeline_name="strava_rest_api_from_config",
    destination="duckdb",
    #dataset_name="rest_api_data",
    dev_mode=True,
)

load_info = pipeline_from_config.run(strava_source)

In [14]:
conn = duckdb.connect(f"strava_rest_api_from_config.duckdb")
conn.sql(f"SET search_path = 'strava_rest_api_from_config'")
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,activities,"[resource_state, athlete__id, athlete__resourc...","[BIGINT, BIGINT, BIGINT, VARCHAR, DOUBLE, BIGI...",False
4,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,activities__end_latlng,"[value, _dlt_parent_id, _dlt_list_idx, _dlt_id]","[DOUBLE, VARCHAR, BIGINT, VARCHAR]",False
5,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,activities__start_latlng,"[value, _dlt_parent_id, _dlt_list_idx, _dlt_id]","[DOUBLE, VARCHAR, BIGINT, VARCHAR]",False
6,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
7,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
8,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
9,strava_rest_api_from_config,strava_rest_api_from_config_dataset_2025052702...,activities,"[resource_state, athlete__id, athlete__resourc...","[BIGINT, BIGINT, BIGINT, VARCHAR, DOUBLE, BIGI...",False
